In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset,Dataset
# use process bar tool
from tqdm import tqdm
from PIL import Image


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-final-dataset/dataset/test/adults/20.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/6.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/76.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/71.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/5.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/8.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/84.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/85.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/67.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/82.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/30.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/10.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/0.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/62.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/61.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/73.jpg
/kaggle/input/ml-final-dataset/dataset/test/adults/60.jpg
/kaggle/input/ml-f

In [11]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [24]:
aug_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=10), 
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label
    
def list_image_paths(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# List image paths for each class
class_0_image_paths = list_image_paths('/kaggle/input/ml-final-dataset/dataset/train/adults')
class_1_image_paths = list_image_paths('/kaggle/input/ml-final-dataset/dataset/train/children')
class_0_labels = [0] * len(class_0_image_paths)
class_1_labels = [1] * len(class_1_image_paths)

# Combine both classes' image paths and labels
all_image_paths = class_0_image_paths + class_1_image_paths
all_labels = class_0_labels + class_1_labels

original_dataset = CustomDataset(image_paths=all_image_paths, labels=all_labels, transform=None)

def generate_augmented_dataset(original_dataset, num_augmented_images_per_class, transform):
    augmented_images = []
    augmented_labels = []
    for label in [0, 1]:
        counter = 0
        while counter < num_augmented_images_per_class:
            for img, lbl in original_dataset:
                if lbl == label:
                    augmented_img = transform(img)
                    augmented_images.append(transforms.ToPILImage(augmented_img))
                    augmented_labels.append(lbl)
                    counter += 1
                    if counter >= num_augmented_images_per_class:
                        break
    return augmented_images, augmented_labels

# Number of augmented image generated
num_augmented_images_per_class = 720
augmented_images, augmented_labels = generate_augmented_dataset(original_dataset, num_augmented_images_per_class, aug_transforms)

combined_images = all_image_paths + augmented_images
combined_labels = all_labels + augmented_labels

combined_dataset = CustomDataset(image_paths=combined_images, labels=combined_labels, transform = None)
train_loader = DataLoader(combined_dataset, batch_size=32, shuffle=True)

In [25]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
#train_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/kaggle/input/ml-final-dataset/dataset/test', transform=transform)

#train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [26]:
# Load pre-trained EfficientNet model
model = models.efficientnet_b0(pretrained=True)
#for param in model.features.parameters():
#    param.requires_grad = False
#model = torch.load('/kaggle/working/2024_06_15_trial.pt')

# Modify the final layer to match the number of classes in the final project
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, 2)  # Binary classification (2 classes)

# Transfer model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [27]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Use tqdm to create a progress bar for the data loader
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch"):
        images, labels = images.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')


Epoch 1/20:   0%|          | 0/63 [00:00<?, ?batch/s]


AttributeError: 'ToPILImage' object has no attribute 'read'

In [11]:
# Evaluate the model on the test dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Test Accuracy: 87.50%


In [10]:
# Evaluate the model on the train dataset
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Train Accuracy: {100 * correct / total:.2f}%')

Train Accuracy: 99.82%


In [19]:
%pip install torchprofile torchsummary

Note: you may need to restart the kernel to use updated packages.


In [20]:
from torchsummary import summary
import torchvision.models as models
from torchprofile import profile_macs
# Calculate the number of trainable parameters and all parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())

print('trainable_params = ', trainable_params)
print('all_params = ', all_params)

# Create a dummy input tensor with the same size as the input images
input_tensor = torch.randn(1, 3, 224, 224).to(device)

# Calculate FLOPs
flops = profile_macs(model, input_tensor)
print(f'FLOPs: {flops / 1e9:.2f} GFLOPs')  # Convert to GFLOPs (GigaFLOPs)

trainable_params =  4010110
all_params =  4010110
FLOPs: 0.39 GFLOPs


/opt/conda/lib/python3.10/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::silu_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [9]:
torch.save(model, '/kaggle/working/2024_06_15_trial.pt')

In [21]:
#print("Model Layers:")
#omodel = models.efficientnet_b0(pretrained=True)
for layer in model.children():
    print(layer)

Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), 